## Grad-CAM Evrişimsel Sinir Ağları tekniğinin aptos datalarına uygulanması

***TEZ İÇİN ÖNEMLİ -> ALTTA YER ALAN FLOW CHART İŞLENECEK!

!(http://gradcam.cloudcv.org/static/images/network.png)

Grad-CAM görselleştirme şu şekilde özetlenebilir:

**Amaç** Modelin nihai öngörülen sınıf (burada, diyabetik retinopati şiddet düzeyi) hakkında karar vermesini sağlayan piksel bölgelerini (uzaysal bilgi) vurgulanması şeklindedir. Bu bölgeleri **ısı haritası** kullanarak görselleştiriyoruz (yukarıdaki şekilde gösterildiği gibi).

**Yöntem Sezgisi**
* En önemli uzamsal bilginin, son FC katmanına akan en yakın uzamsal bilgi olan *son evrişim katmanının* ('GlobalPooling katmanı'ndan hemen önce) 3B tensöründen geldiği görülmekte.

* Bu 3D tensörün her kanalı için, etkinleştirilmiş her piksel bölgesi, giriş görüntüsünün önemli özelliklerini (örneğin kan damarı / kabuk / cotton noktası vbb.) temsil eder. Bazı özelliklerin sınıf 0'ı (mükemmel ince kan damarı) belirlemek için önemli olduğunu, bazı özelliklerin sınıf 4'ü (büyük ebatlı cotton noktaları) belirlemek için önemli olduğunu unutmayın. Normalde, her kanalın farklı özellikler dizisi yakalamasını bekleriz.

* Nihai tahmini etkileyen özellikleri vurgulamak için, **son tahmin edilen sınıfın her bir özelliğe göre gradyanını hesaplarız.** Bu özellik bu sınıf için önemliyse, yüksek eğime sahip olmalıdır (yani değerini artırın). bu özellik, prediction'ın yani kestirimin güvenini arttırır)

* Bu nedenle, her kanal için görselleştirilmiş ısı haritasını elde etmek için bu 3B tensörün etkinleştirilmiş değerlerini ve gradyanları birlikte çarpıyoruz. Çoklu kanallarımız olduğunu ve her kanalın genellikle çoklu özellikleri olduğunu unutmayın.

* Son olarak, basit bir ortalama kullanarak tüm kanalların ısı haritalarını birleştirir ve nihai ısı haritasını elde etmek için negatif değeri (yukarıdaki resimde 'ReLu' adımı) kaldırılmaktadır. 



In [ ]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm

%matplotlib inline

Model çıktısını Grad-CAM kullanarak görselleştirmek için modeli yeniden yüklenmesine gerek bulunmamakta, zira depo içeriisnde aptos datalarını bir önceki ŞY-1 noktasından çekmekteyiz. 

**Orijinal modülün 224x224 görüntü kullandığını ve aşağıdaki preprocess_image işlevini kullandığını unutmayın.

Yine de, model ağırlıklarını doğru yüklediğimizden emin olmak için test verilerini önceden işleyeceğim ve orijinal çekirdekle tam olarak aynı tahminleri elde ettiğimizden emin oluyoruz.

In [ ]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
print(train_df.shape)
print(test_df.shape)
test_df.head()

In [ ]:
def get_pad_width(im, new_shape, is_rgb=True):
    pad_diff = new_shape - im.shape[0], new_shape - im.shape[1]
    t, b = math.floor(pad_diff[0]/2), math.ceil(pad_diff[0]/2)
    l, r = math.floor(pad_diff[1]/2), math.ceil(pad_diff[1]/2)
    if is_rgb:
        pad_width = ((t,b), (l,r), (0, 0))
    else:
        pad_width = ((t,b), (l,r))
    return pad_width

def preprocess_image(image_path, desired_size=224):
    im = Image.open(image_path)
    im = im.resize((desired_size, )*2, resample=Image.LANCZOS)
#     im = im.resize((desired_size, )*2)
    
    return im

In [ ]:
# test datalarının sisteme yüklenmesi, png verileri daha nitelikli olduğundan dolayı sisteme yüklüyoruz!

N = test_df.shape[0]
x_test = np.empty((N, 224, 224, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(test_df['id_code'])):
    x_test[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/test_images/{image_id}.png'
    )

### Bazı orijinal test görüntülerini görüntüleme

Sadece bir fikir olması için, test setindeki ilk 10 gözü görselleştirelim. Burada az sayıda gözün iyi göründüğünü gözlenmekte (tez içerisinde önemli olduğu vurgulanacak). tıp uygulamaları için anormal noktaları daha kolay göreceğimiz ve daha sonra göz resmini ısı haritası ile birleştirdiğimizde daha temiz olacağı için ŞY-1 uygulamasında yer alan önişlemeleri çağırıyoruz 

In [ ]:
# model.summary()
def load_image_ben_orig(path,resize=True,crop=False,norm255=True,keras=False):
    image = cv2.imread(path)
    
#     if crop:
#         image = crop_image(image)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
#     if resize:
#         image = cv2.resize(image,(SIZE,SIZE))
        
    image=cv2.addWeighted( image,4, cv2.GaussianBlur( image , (0,0) ,  10) ,-4 ,128)
#     image=cv2.addWeighted( image,4, cv2.medianBlur( image , 10) ,-4 ,128)
    
    # NOTE plt.imshow can accept both int (0-255) or float (0-1), but deep net requires (0-1)
    if norm255:
        return image/255
    elif keras:
        #see https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py for mode
        #see https://github.com/keras-team/keras-applications/blob/master/keras_applications/xception.py for inception,xception mode
        #the use of tf based preprocessing (- and / by 127 respectively) will results in [-1,1] so it will not visualize correctly (directly)
        image = np.expand_dims(image, axis=0)
        return preprocess_input(image)[0]
    else:
        return image.astype(np.int16)
    
    return image

def transform_image_ben(img,resize=True,crop=False,norm255=True,keras=False):  
    image=cv2.addWeighted( img,4, cv2.GaussianBlur( img , (0,0) ,  10) ,-4 ,128)
    
    # NOTE plt.imshow can accept both int (0-255) or float (0-1), but deep net requires (0-1)
    if norm255:
        return image/255
    elif keras:
        image = np.expand_dims(image, axis=0)
        return preprocess_input(image)[0]
    else:
        return image.astype(np.int16)
    
    return image

In [ ]:
def display_samples(df, columns=5, rows=2, Ben=True):
    fig=plt.figure(figsize=(5*columns, 4*rows))

    for i in range(columns*rows):
        image_path = df.loc[i,'id_code']
#         image_id = df.loc[i,'diagnosis']
        path = f'../input/aptos2019-blindness-detection/test_images/{image_path}.png'
        if Ben:
            img = load_image_ben_orig(path)
        else:
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        fig.add_subplot(rows, columns, i+1)
#         plt.title(image_id)
        plt.imshow(img)
    
    plt.tight_layout()

display_samples(test_df, Ben=False)
display_samples(test_df, Ben=True)

# 3. Modeli oluşturma konulu gradyan çıktı modülü

Kavramsal olarak, sadece önceden eğitilmiş modeli yükleyebilir ve istenen gradyanı hesaplayabiliriz ve bu, ısı haritasına sahip olmak için yeterlidir! Ancak, teknik olarak konuşursak, orijinal çekirdek Keras'ın 'İşlevsel yöntemi' yerine ince ayarlı bir DenseNet modeli oluşturmak için Keras'ın 'Sıralı yöntemini' kullanır.Paylaşılan katmanları kullanarak bir model oluşturmak için 'Sıralı yöntemi' kullanmak ve ardından önceden eğitilmiş ağırlıklar uygulamaktır. Bundan sonra, `Fonksiyonel yöntem` kullanarak ancak aynı paylaşılan katmanları kullanarak başka bir model oluşturuyoruz. Tüm katmanlar paylaşıldığından, iki model tamamen aynıdır, aynı ağırlıklara sahiptir.


keras ile derin öğrenme modeli -> https://medium.com/@tuncerergin/keras-ile-derin-ogrenme-modeli-olusturma-4b4ffdc35323

In [ ]:
from keras import layers
from keras.models import Model
import keras.backend as K

In [ ]:
K.clear_session()
densenet = DenseNet121(
    weights=None,
    include_top=False,
    input_shape=(None,None,3)
)

Daha sonra, orijinal çekirdekte kullanılanla tamamen aynı türde 3 paylaşılan kafa katmanı tanımlıyoruz. Ardından, orijinal çekirdekle aynı işlemi `Sequential()` modülünü kullanarak oluşturun. Aşağıdaki `model.summary()`den `Sequential()` modülünü kullanarak omurganın katman detaylarının gizlendiğini ve doğrudan kullanamadığımızı görebilirsiniz. **tezde belirtilecek ayrı bir detay.

In [ ]:
GAP_layer = layers.GlobalAveragePooling2D()
drop_layer = layers.Dropout(0.5)
dense_layer = layers.Dense(5, activation='sigmoid', name='final_output')

In [ ]:
def build_model_sequential():
    model = Sequential()
    model.add(densenet)
    model.add(GAP_layer)
    model.add(drop_layer)
    model.add(dense_layer)
    return model

In [ ]:
modelA = build_model_sequential()
modelA.load_weights('../input/aptos-data/dense_xhlulu_731.h5')

modelA.summary()

Below, we construct another model using exactly the same (shared) layers. When pretrained weights are loaded into the first model, the second model also get the same weights (since all layers are shared)

In [ ]:
def build_model_functional():
    base_model = densenet
    
    x = GAP_layer(base_model.layers[-1].output)
    x = drop_layer(x)
    final_output = dense_layer(x)
    model = Model(base_model.layers[0].input, final_output)
    
    return model

Artık fonksiyonel modülü kullanarak, `model.summary()` çalıştırıldığında görünen tüm katmanlara erişebiliriz. Çıktı çok uzun olduğu için arasıra bu arayüz kitlenebilir, tekrardan deneyiniz !

In [ ]:
model = build_model_functional() 
model.summary()

Bu nedenle, son evrişim katmanına buradan erişebiliriz. "conv5_block16_concat"ın düzeltilmiş ve toplu normalleştirilmiş sürümü olan "conv5_block16_concat" veya "relu" kullanabileceğimizi unutmayın.

## 3.1 Sadece doğru ağırlıklara sahip olduğumuzdan emin olmak için

Bu bölümün amacının, zaten doğru ağırlıkları yüklediğimizden emin olmak olduğunu unutmayın. ÖLB puanı ile kanıtlandığı için bu sürümde yorumlamak mümkündür.

In [ ]:
# y_test = model.predict(x_test) > 0.5
# y_test = y_test.astype(int).sum(axis=1) - 1

# test_df['diagnosis'] = y_test
# test_df.to_csv('submission.csv',index=False)
# y_test.shape, x_test.shape

In [ ]:
# import seaborn as sns
# import cv2

# SIZE=224
# def create_pred_hist(pred_level_y,title='NoTitle'):
#     results = pd.DataFrame({'diagnosis':pred_level_y})

#     f, ax = plt.subplots(figsize=(7, 4))
#     ax = sns.countplot(x="diagnosis", data=results, palette="GnBu_d")
#     sns.despine()
#     plt.title(title)
#     plt.show()

In [ ]:
# create_pred_hist(y_test,title='predicted level distribution in test set')   #-> kestirim seviyesinin belirlenmesi 

# 4. Gerçek veya Sanal Anomalilerin tespiti

Önce bir ısı haritası hesaplama fonksiyonu tanımlayalım. Giriş bölümünde belirtildiği gibi, kodlar [bu makaleden](http://www.hackevolve.com/where-cnn-is-look-grad-cam/) uyarlanmıştır ve bu da F.Chollet'in kitabından uyarlanmıştır.

Bu fonksiyon girdi olarak 4 argüman alacaktır. (1) bir tahmin/görselleştirme yapmak için görüntü, buraya doğru önceden işlenmiş versiyonu eklemeyi unutmayın (2) model (3) gradyanları elde etmek için bir katman ve (4) sadece ısı haritası ile birleştirmek ve nihai sonucu görselleştirmek için bir yardımcı görüntü ; Resimlerdeki yıldırım koşullarını ortadan kaldırdığı ve nihai sonucu görselleştirmemiz amacıyla ŞY_1 scriptinde yer alan işlenmiş görüntüleri kullanmaktayız.

In [ ]:
def gen_heatmap_img(img, model0, layer_name='last_conv_layer',viz_img=None,orig_img=None):
    preds_raw = model0.predict(img[np.newaxis])
    preds = preds_raw > 0.5 
    class_idx = (preds.astype(int).sum(axis=1) - 1)[0]
#     print(class_idx, class_idx.shape)
    class_output_tensor = model0.output[:, class_idx]
    
    viz_layer = model0.get_layer(layer_name)
    grads = K.gradients(
                        class_output_tensor ,
                        viz_layer.output
                        )[0] 
    
    pooled_grads=K.mean(grads,axis=(0,1,2))
    iterate=K.function([model0.input],[pooled_grads, viz_layer.output[0]])
    
    pooled_grad_value, viz_layer_out_value = iterate([img[np.newaxis]])
    
    for i in range(pooled_grad_value.shape[0]):
        viz_layer_out_value[:,:,i] *= pooled_grad_value[i]
    
    heatmap = np.mean(viz_layer_out_value, axis=-1)
    heatmap = np.maximum(heatmap,0)
    heatmap /= np.max(heatmap)

    viz_img=cv2.resize(viz_img,(img.shape[1],img.shape[0]))
    heatmap=cv2.resize(heatmap,(viz_img.shape[1],viz_img.shape[0]))
    
    heatmap_color = cv2.applyColorMap(np.uint8(heatmap*255), cv2.COLORMAP_SPRING)/255
    heated_img = heatmap_color*0.5 + viz_img*0.5
    
    print('raw output from model : ')
    print_pred(preds_raw)
    
    if orig_img is None:
        show_Nimages([img,viz_img,heatmap_color,heated_img])
    else:
        show_Nimages([orig_img,img,viz_img,heatmap_color,heated_img])
    
    plt.show()
    return heated_img

In [ ]:
def show_image(image,figsize=None,title=None):
    
    if figsize is not None:
        fig = plt.figure(figsize=figsize)
#     else: # crash!!
#         fig = plt.figure()
        
    if image.ndim == 2:
        plt.imshow(image,cmap='gray')
    else:
        plt.imshow(image)
        
    if title is not None:
        plt.title(title)

def show_Nimages(imgs,scale=1):

    N=len(imgs)
    fig = plt.figure(figsize=(25/scale, 16/scale))
    for i, img in enumerate(imgs):
        ax = fig.add_subplot(1, N, i + 1, xticks=[], yticks=[])
        show_image(img)
        
def print_pred(array_of_classes):
    xx = array_of_classes
    s1,s2 = xx.shape
    for i in range(s1):
        for j in range(s2):
            print('%.3f ' % xx[i,j],end='')
        print('')

In [ ]:
NUM_SAMP=10
SEED=77
layer_name = 'relu' #'conv5_block16_concat'
for i, (idx, row) in enumerate(test_df[:NUM_SAMP].iterrows()):
    path=f"../input/aptos2019-blindness-detection/test_images/{row['id_code']}.png"
    ben_img = load_image_ben_orig(path)
    input_img = np.empty((1,224, 224, 3), dtype=np.uint8)
    input_img[0,:,:,:] = preprocess_image(path)
        
    print('test pic no.%d' % (i+1))
    _ = gen_heatmap_img(input_img[0],
                        model, layer_name=layer_name,viz_img=ben_img)

Burada birçok ilginç gözlem var. Birkaç isim,

* 1., 4., 5., 6. tahminler harika görünüyor
* 2. tahmin, ortadaki tüm büyük noktaları kaçırır
* 3. ve 7. tahminler de önemli noktaları kaçırıyor
* 9. görselde kanlı noktalar her yerde doğru tespitler bulunmakta.

# 5. Sağlamlık Testi



In [ ]:
from albumentations import *
import time

IMG_SIZE = (224,224)


def albaugment(aug0, img):
    return aug0(image=img)['image']
idx=8
image1=x_test[idx]

'''1. Rotate or Flip'''
aug1 = OneOf([
    Rotate(p=0.99, limit=160, border_mode=0,value=0), 
    Flip(p=0.5)
    ],p=1)

'''2. Adjust Brightness or Contrast'''
aug2 = RandomBrightnessContrast(brightness_limit=0.45, contrast_limit=0.45,p=1)
h_min=np.round(IMG_SIZE[1]*0.72).astype(int)
h_max= np.round(IMG_SIZE[1]*0.9).astype(int)
print(h_min,h_max)

'''3. Random Crop and then Resize'''
#w2h_ratio = aspect ratio of cropping
aug3 = RandomSizedCrop((h_min, h_max),IMG_SIZE[1],IMG_SIZE[0], w2h_ratio=IMG_SIZE[0]/IMG_SIZE[1],p=1)

'''4. CutOut Augmentation'''
max_hole_size = int(IMG_SIZE[1]/10)
aug4 = Cutout(p=1,max_h_size=max_hole_size,max_w_size=max_hole_size,num_holes=8 )#default num_holes=8

'''5. SunFlare Augmentation'''
aug5 = RandomSunFlare(src_radius=max_hole_size,
                      num_flare_circles_lower=10,
                      num_flare_circles_upper=20,
                      p=1)#default flare_roi=(0,0,1,0.5),

'''6. Ultimate Augmentation -- combine everything'''
final_aug = Compose([
    aug1,aug2,aug3,aug4,aug5
],p=1)


img1 = albaugment(aug1,image1)
img2 = albaugment(aug1,image1)
print('Rotate or Flip')
show_Nimages([image1,img1,img2],scale=2)
# time.sleep(1)

img1 = albaugment(aug2,image1)
img2 = albaugment(aug2,image1)
img3 = albaugment(aug2,image1)
print('Brightness or Contrast')
show_Nimages([img3,img1,img2],scale=2)
# time.sleep(1)

img1 = albaugment(aug3,image1)
img2 = albaugment(aug3,image1)
img3 = albaugment(aug3,image1)
print('Rotate and Resize')
show_Nimages([img3,img1,img2],scale=2)
print(img1.shape,img2.shape)
# time.sleep(1)

img1 = albaugment(aug4,image1)
img2 = albaugment(aug4,image1)
img3 = albaugment(aug4,image1)
print('CutOut')
show_Nimages([img3,img1,img2],scale=2)
# time.sleep(1)

img1 = albaugment(aug5,image1)
img2 = albaugment(aug5,image1)
img3 = albaugment(aug5,image1)
print('Sun Flare')
show_Nimages([img3,img1,img2],scale=2)
# time.sleep(1)

img1 = albaugment(final_aug,image1)
img2 = albaugment(final_aug,image1)
img3 = albaugment(final_aug,image1)
print('All above combined')
show_Nimages([img3,img1,img2],scale=2)
print(img1.shape,img2.shape)


Büyütme rastgele olduğu için, bu çekirdeği yazdığımdan farklı sonuçlar göreceksiniz. Denememde, model oldukça sağlam çünkü bazen kafa karıştırıcı bir şekilde seviye 4 olarak tahmin ettiği nihai (her şey) büyütme dışında neredeyse aynı seviyeyi tahmin ediyor. Algılanan özellikler de oldukça tutarlı.



In [ ]:
aug_list = [aug5, aug2, aug3, aug4, aug1, final_aug]
aug_name = ['SunFlare', 'brightness or contrast', 'crop and resized', 'CutOut', 'rotate or flip', 'Everything Combined']

idx=8
layer_name = 'relu' #'conv5_block16_concat'
for i in range(len(aug_list)):
    path=f"../input/aptos2019-blindness-detection/test_images/{test_df.iloc[idx]['id_code']}.png"
    input_img = np.empty((1,224, 224, 3), dtype=np.uint8)
    input_img[0,:,:,:] = preprocess_image(path)
    aug_img = albaugment(aug_list[i],input_img[0,:,:,:])
    ben_img = transform_image_ben(aug_img)
    
    print('test pic no.%d -- augmentation: %s' % (i+1, aug_name[i]))
    _ = gen_heatmap_img(aug_img,
                        model, layer_name=layer_name,viz_img=ben_img,orig_img=input_img[0])

Gelecekte güncelleyeceğim bu ısı haritası görselleştirmesinin daha birçok olası yaratıcı kullanımı var.

* Artırılmış görüntüleri görselleştirin ve modelimizin yeterince sağlam olup olmadığına (aynı şeyi tahmin ediyor) veya büyütmemizin anlamlı olup olmadığına (önemli bilgileri koruyor mu?) tez içerisinde bakılmalı

* Modelimizin her bir önem derecesini belirlemek için hangi özellikleri kullandığı hakkında bir fikir edinmek için 0'dan 4'e kadar her seviye için görselleştirin
